In [1]:
import mne
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from mne.decoding import SlidingEstimator, cross_val_multiscore
from bids import BIDSLayout
from mne.minimum_norm import (apply_inverse, apply_inverse_epochs,
                              read_inverse_operator)
from mne_connectivity import seed_target_indices, spectral_connectivity_epochs

In [2]:
BIDS_ROOT = '../data/bids'
FIGS_ROOT = '../figs'
DERIV_ROOT = '../data/bids/derivatives'
FS = 5000
FREQS = [50, 100, 150, 200, 250]

In [3]:
layout = BIDSLayout(BIDS_ROOT, derivatives = True)
fpaths = layout.get(scope = 'preprocessing',
                    res = 'hi',
                    suffix='epo',
                    extension = 'fif.gz',
                    return_type = 'filename')

/home/letitiayhho/.conda/envs/pitch-tracking/lib/python3.10/site-packages/bids/layout/validation.py:151: UserWarning: The PipelineDescription field was superseded by GeneratedBy in BIDS 1.4.0. You can use ``pybids upgrade`` to update your derivative dataset.
  warnings.warn("The PipelineDescription field was superseded "


In [19]:
epochs = mne.read_epochs(fpaths[10])
events = mne.read_events(fpaths[10])
n_epochs = len(events)

Reading /project2/hcn1/pitch_tracking/scripts/../data/bids/derivatives/preprocessing/sub-9/sub-9_task-pitch_run-1_res-hi_desc-clean_epo.fif.gz ...
    Found the data of interest:
        t =    -200.00 ...     250.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
1922 matching events found
No baseline correction applied
0 projection items activated


/tmp/ipykernel_45564/1587172011.py:2: RuntimeWarning: This filename (/project2/hcn1/pitch_tracking/scripts/../data/bids/derivatives/preprocessing/sub-9/sub-9_task-pitch_run-1_res-hi_desc-clean_epo.fif.gz) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst, _eve.txt or -annot.fif
  events = mne.read_events(fpaths[10])


### Simulate channels with stim signals

In [5]:
# Create 200 msec sine waves at condition frequencies
tmin = -0.2
tmax = 0.25
samples_in_epoch = int(FS*(abs(tmin) + tmax))+1
times = np.linspace(-0.2, 0.25, samples_in_epoch, endpoint = False) # times for a 200 msec epoch
sine_50 = np.sin(50 * 2 * np.pi * times)
sine_100 = np.sin(100 * 2 * np.pi * times)
sine_150 = np.sin(150 * 2 * np.pi * times)
sine_200 = np.sin(200 * 2 * np.pi * times)
sine_250 = np.sin(250 * 2 * np.pi * times)

# Broadcast sine waves by number of epochs
sine_50_epochs = [sine_50]*n_epochs
sine_100_epochs = [sine_100]*n_epochs
sine_150_epochs = [sine_150]*n_epochs
sine_200_epochs = [sine_200]*n_epochs
sine_250_epochs = [sine_250]*n_epochs

# Combine sine waves into array
data = np.stack((sine_50_epochs, 
                 sine_100_epochs,
                 sine_150_epochs,
                 sine_200_epochs,
                 sine_250_epochs,
                ), axis = 1)
print(np.shape(data)) # n_epochs, n_channels, n_samples

# Specify channel info
info = mne.create_info(ch_names = ['50haz', 
                                   '100hz',
                                   '150hz',
                                   '200hz',
                                   '250hz'],
                       ch_types = ['stim'] * 5,
                       sfreq = FS)

# Manually add channel info to match original data to stop mne from shouting at us, very hacky
info['custom_ref_applied'] = True
info['description'] = 'Anonymized using a time shift to preserve age at acquisition'
info['experimenter'] = 'mne_anonymize'
info['highpass'] = 30.0
info['line_freq'] = 60.0
info['lowpass'] = 270.0
event_id = {'100': 10001, '150': 10002, '200': 10003, '250': 10004, '50': 10005}

# Manually add info that is passed in through mne.EpochsArray instead of in the info dict, also very hacky
tmin = -0.2
tmax = 0.25
baseline = (-0.20000000298023224, 0.0)

# Create Epochs object
simulated_epochs = mne.EpochsArray(data, 
                                   info, 
                                   events = events, 
                                   tmin = tmin, 
                                   event_id = event_id, 
                                   baseline = baseline)

(1922, 5, 2251)
Not setting metadata
1922 matching events found
Applying baseline correction (mode: mean)
0 projection items activated


/home/letitiayhho/.conda/envs/pitch-tracking/lib/python3.10/site-packages/mne/io/meas_info.py:864: UserWarning: Avoid manually setting custom_ref_applied cannot be set directly. Please use method inst.set_eeg_reference() instead.
  warnings.warn(f"Avoid manually setting {self._attributes[key]}")
/home/letitiayhho/.conda/envs/pitch-tracking/lib/python3.10/site-packages/mne/io/meas_info.py:864: UserWarning: Avoid manually setting highpass cannot be set directly. Please use method inst.filter() instead.
  warnings.warn(f"Avoid manually setting {self._attributes[key]}")
/home/letitiayhho/.conda/envs/pitch-tracking/lib/python3.10/site-packages/mne/io/meas_info.py:864: UserWarning: Avoid manually setting lowpass cannot be set directly. Please use method inst.filter() instead.
  warnings.warn(f"Avoid manually setting {self._attributes[key]}")


0 bad epochs dropped


In [6]:
# Combine original channels with simulated channels
combined_epochs = mne.epochs.add_channels_epochs([epochs, simulated_epochs])

### Compute coherence

In [41]:
data = combined_epochs.get_data()
fmin = (45, 95, 145, 195, 245)
fmax = (55, 105, 155, 205, 255)

In [44]:
# Set indices of channel pairs to compute coherence across
stim_indices = np.array([62, 63, 64, 65, 66]*62)
chan_indices = np.repeat(np.arange(0, 63, 1), 5)
print(stim_indices)
print(chan_indices)

[62 63 64 65 66 62 63 64 65 66 62 63 64 65 66 62 63 64 65 66 62 63 64 65
 66 62 63 64 65 66 62 63 64 65 66 62 63 64 65 66 62 63 64 65 66 62 63 64
 65 66 62 63 64 65 66 62 63 64 65 66 62 63 64 65 66 62 63 64 65 66 62 63
 64 65 66 62 63 64 65 66 62 63 64 65 66 62 63 64 65 66 62 63 64 65 66 62
 63 64 65 66 62 63 64 65 66 62 63 64 65 66 62 63 64 65 66 62 63 64 65 66
 62 63 64 65 66 62 63 64 65 66 62 63 64 65 66 62 63 64 65 66 62 63 64 65
 66 62 63 64 65 66 62 63 64 65 66 62 63 64 65 66 62 63 64 65 66 62 63 64
 65 66 62 63 64 65 66 62 63 64 65 66 62 63 64 65 66 62 63 64 65 66 62 63
 64 65 66 62 63 64 65 66 62 63 64 65 66 62 63 64 65 66 62 63 64 65 66 62
 63 64 65 66 62 63 64 65 66 62 63 64 65 66 62 63 64 65 66 62 63 64 65 66
 62 63 64 65 66 62 63 64 65 66 62 63 64 65 66 62 63 64 65 66 62 63 64 65
 66 62 63 64 65 66 62 63 64 65 66 62 63 64 65 66 62 63 64 65 66 62 63 64
 65 66 62 63 64 65 66 62 63 64 65 66 62 63 64 65 66 62 63 64 65 66]
[ 0  0  0  0  0  1  1  1  1  1  2  2  2  2  2  3  3  3  

In [43]:
coh = spectral_connectivity_epochs(
    data, method='coh', mode='fourier', indices=indices,
    sfreq=FS, fmin=fmin, fmax=fmax, faverage=True, n_jobs=1)
freqs = coh.freqs

print('Frequencies in Hz over which coherence was averaged for 50 Hz: ')
print(freqs[0])
print('Frequencies in Hz over which coherence was averaged for 100 Hz: ')
print(freqs[1])
print('Frequencies in Hz over which coherence was averaged for 150 Hz: ')
print(freqs[2])
print('Frequencies in Hz over which coherence was averaged for 200 Hz: ')
print(freqs[3])
print('Frequencies in Hz over which coherence was averaged for 250 Hz: ')
print(freqs[4])

Connectivity computation...
    computing connectivity for 7498 connections
    using t=0.000s..0.450s for estimation (2251 points)
    computing connectivity for the bands:
     band 1: 46.6Hz..53.3Hz (4 points)
     band 2: 95.5Hz..104.4Hz (5 points)
     band 3: 146.6Hz..153.3Hz (4 points)
     band 4: 195.5Hz..204.4Hz (5 points)
     band 5: 246.6Hz..253.2Hz (4 points)
    connectivity scores will be averaged for each band
    using FFT with a Hanning window to estimate spectra
    the following metrics will be computed: Coherence
    computing connectivity for epoch 1


IndexError: index 67 is out of bounds for axis 0 with size 67

In [9]:

import numpy as np

import mne
from mne.datasets import sample
from mne.minimum_norm import (apply_inverse, apply_inverse_epochs,
                              read_inverse_operator)
from mne_connectivity import seed_target_indices, spectral_connectivity_epochs

print(__doc__)

Automatically created module for IPython interactive environment


In [10]:
data_path = sample.data_path()
subjects_dir = data_path + '/subjects'
fname_inv = data_path + '/MEG/sample/sample_audvis-meg-oct-6-meg-inv.fif'
fname_raw = data_path + '/MEG/sample/sample_audvis_filt-0-40_raw.fif'
fname_event = data_path + '/MEG/sample/sample_audvis_filt-0-40_raw-eve.fif'
label_name_lh = 'Aud-lh'
fname_label_lh = data_path + '/MEG/sample/labels/%s.label' % label_name_lh

event_id, tmin, tmax = 1, -0.2, 0.5
method = "dSPM"  # use dSPM method (could also be MNE or sLORETA)

# Load data.
inverse_operator = read_inverse_operator(fname_inv)
label_lh = mne.read_label(fname_label_lh)
raw = mne.io.read_raw_fif(fname_raw)
events = mne.read_events(fname_event)

# Add a bad channel.
raw.info['bads'] += ['MEG 2443']

# pick MEG channels.
picks = mne.pick_types(raw.info, meg=True, eeg=False, stim=False, eog=True,
                       exclude='bads')

# Read epochs.
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, picks=picks,
                    baseline=(None, 0),
                    reject=dict(mag=4e-12, grad=4000e-13, eog=150e-6))

Using default location ~/mne_data for sample...
Creating ~/mne_data


100%|█████████████████████████████████████| 1.65G/1.65G [00:00<00:00, 1.26TB/s]
Untarring contents of '/home/letitiayhho/mne_data/MNE-sample-data-processed.tar.gz' to '/home/letitiayhho/mne_data'


Attempting to create new mne-python configuration file:
/home/letitiayhho/.mne/mne-python.json
Reading inverse operator decomposition from /home/letitiayhho/mne_data/MNE-sample-data/MEG/sample/sample_audvis-meg-oct-6-meg-inv.fif...
    Reading inverse operator info...
    [done]
    Reading inverse operator decomposition...
    [done]
    305 x 305 full covariance (kind = 1) found.
    Read a total of 4 projection items:
        PCA-v1 (1 x 102) active
        PCA-v2 (1 x 102) active
        PCA-v3 (1 x 102) active
        Average EEG reference (1 x 60) active
    Noise covariance matrix read.
    22494 x 22494 diagonal covariance (kind = 2) found.
    Source covariance matrix read.
    22494 x 22494 diagonal covariance (kind = 6) found.
    Orientation priors read.
    22494 x 22494 diagonal covariance (kind = 5) found.
    Depth priors read.
    Did not find the desired covariance matrix (kind = 3)
    Reading a source space...
    Computing patch statistics...
    Patch information 

/tmp/ipykernel_45564/13216504.py:2: DeprecationWarning: data_path functions now return pathlib.Path objects which do not natively support the plus (+) operator, switch to using forward slash (/) instead. Support for plus will be removed in 1.2.
  subjects_dir = data_path + '/subjects'
/tmp/ipykernel_45564/13216504.py:3: DeprecationWarning: data_path functions now return pathlib.Path objects which do not natively support the plus (+) operator, switch to using forward slash (/) instead. Support for plus will be removed in 1.2.
  fname_inv = data_path + '/MEG/sample/sample_audvis-meg-oct-6-meg-inv.fif'
/tmp/ipykernel_45564/13216504.py:4: DeprecationWarning: data_path functions now return pathlib.Path objects which do not natively support the plus (+) operator, switch to using forward slash (/) instead. Support for plus will be removed in 1.2.
  fname_raw = data_path + '/MEG/sample/sample_audvis_filt-0-40_raw.fif'
/tmp/ipykernel_45564/13216504.py:5: DeprecationWarning: data_path functions 

Not setting metadata
72 matching events found
Setting baseline interval to [-0.19979521315838786, 0.0] sec
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 3)
4 projection items activated


In [11]:
snr = 3.0
lambda2 = 1.0 / snr ** 2
evoked = epochs.average()
stc = apply_inverse(evoked, inverse_operator, lambda2, method,
                    pick_ori="normal")

# Restrict the source estimate to the label in the left auditory cortex.
stc_label = stc.in_label(label_lh)

# Find number and index of vertex with most power.
src_pow = np.sum(stc_label.data ** 2, axis=1)
seed_vertno = stc_label.vertices[0][np.argmax(src_pow)]
seed_idx = np.searchsorted(stc.vertices[0], seed_vertno)  # index in orig stc

# Generate index parameter for seed-based connectivity analysis.
n_sources = stc.data.shape[0]
indices = seed_target_indices([seed_idx], np.arange(n_sources))

print(indices)

    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on MAG : ['MEG 1711']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
Removing projector <Projection | Average EEG reference, active : True, n_channels : 60>
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 

In [12]:
print(indices)

(array([1702, 1702, 1702, ..., 1702, 1702, 1702]), array([   0,    1,    2, ..., 7495, 7496, 7497]))


In [15]:
type(indices[0])

numpy.ndarray

In [28]:
# so, pairs of channels to compute against... so 62 channels * 5 freqs
# first array contains the stim channels
# np.array([])
mne.pick_types(epochs.info, eeg = True)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61])

In [22]:
epochs.info

<Info | 15 non-empty values
 bads: []
 ch_names: Fp1, Fz, F3, F7, FT9, FC5, FC1, C3, T7, TP9, CP5, CP1, Pz, P3, ...
 chs: 62 EEG
 custom_ref_applied: True
 description: Anonymized using a time shift to preserve age at acquisition
 dig: 62 items (62 EEG)
 experimenter: mne_anonymize
 file_id: 4 items (dict)
 highpass: 30.0 Hz
 line_freq: 60.0
 lowpass: 270.0 Hz
 meas_date: 1924-12-31 15:45:53 UTC
 meas_id: 4 items (dict)
 nchan: 62
 projs: []
 sfreq: 5000.0 Hz
 subject_info: 3 items (dict)
>

In [23]:
simulated_epochs.info

<Info | 11 non-empty values
 bads: []
 ch_names: 50haz, 100hz, 150hz, 200hz, 250hz
 chs: 5 Stimulus
 custom_ref_applied: True
 description: Anonymized using a time shift to preserve age at acquisition
 experimenter: mne_anonymize
 highpass: 30.0 Hz
 line_freq: 60.0
 lowpass: 270.0 Hz
 meas_date: unspecified
 nchan: 5
 projs: []
 sfreq: 5000.0 Hz
>

In [27]:
mne.pick_types(simulated_epochs.info, stim = True)

array([0, 1, 2, 3, 4])

In [29]:
mne.pick_types(combined_epochs.info, stim = True, eeg = True)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66])